# Had to develop this blocks in another environment.
Please, uncomment the import from other notebook and delete the second block

In [95]:
#from ipynb.fs.defs.MLCP_CLASSIFICATION_3 import train_test, classification_reporting

from sklearn.model_selection import RandomizedSearchCV, train_test_split, cross_val_score
from sklearn.metrics import classification_report, accuracy_score
from sklearn.preprocessing import OneHotEncoder

from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import KFold

from keras.models import Sequential
from keras.layers import Dense, Dropout
import tensorflow as tf

import pandas as pd
import numpy as np

In [96]:
# Define train/test dataset splitting
def train_test(dataset):
    X_train, X_test, y_train, y_test = train_test_split(
        dataset.iloc[:,:dataset.shape[1]-1],
        dataset.iloc[:,-1],
        test_size=0.2,
        stratify=dataset.iloc[:,-1],
        random_state=42)

    return X_train, X_test, y_train, y_test

# Define the classification report. "macro avg" for precision, recall and f1 will be used as a benchmark
def classification_reporting(tuning, model, y_test, y_pred, X_train):
    report = classification_report(y_test, y_pred, output_dict = True)
    print('{} {} performance using {} features:\n'.format(tuning, model, X_train.shape[1]),
          report['macro avg'])
    print('Accuracy score:', accuracy_score(y_test, y_pred))

In [97]:
# Split the dataset with 16 feature
dataset = pd.read_csv('/content/drive/MyDrive/dataset.csv').iloc[:,1:]
X_train_full, X_test_full, y_train_full, y_test_full = train_test(dataset)

y_train_full = pd.get_dummies(y_train_full)
y_test_full = pd.get_dummies(y_test_full)

In [98]:
# Define the model
def model():
  model = Sequential()
  model.add(Dense(32, input_dim=16, activation='relu'))
  model.add(Dropout(0.2))
  model.add(Dense(128, activation='relu'))
  model.add(Dropout(0.2))
  model.add(Dense(64, activation='relu'))
  model.add(Dropout(0.2))
  model.add(Dense(5, activation='softmax'))

  # Compile the model
  model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

  return model

model = model()

In [99]:
model.fit(X_train_full, y_train_full, epochs = 48, batch_size = 10, verbose = 0)

In [100]:
y_pred = model.predict(X_test_full)

y_pred_matrix = pd.DataFrame(np.eye(5)[np.argmax(y_pred, axis = 1)])

classification_reporting('Naive', 'NN', y_test_full, y_pred_matrix, X_train_full)

63/63 [==============================] - 0s 1ms/step
Naive NN performance using 16 features:
 {'precision': 0.5425190631957074, 'recall': 0.5652806966016223, 'f1-score': 0.5280750845517781, 'support': 2000}
Accuracy score: 0.5635
